## Homework 2 - Data Engineering 300
### Aden Benson

#### Part 1: Relational

Note: All csv files are loaded into our homework_2 ec2 instance folder

In [1]:
#!pip install duckdb==1.2.2
import duckdb

In [2]:
# Create duckdb connection
conn=duckdb.connect('hw2.db', read_only=False)

In [3]:
# Add our csv files to the duckdb connection
import pandas as pd
'''
# Add admissions csv
admissions_df=pd.read_csv('hw2_data/ADMISSIONS.csv')
admissions_df.to_sql("ADMISSIONS", conn, index=False)

# Add d_icd_procedures csv
d_icd_procedures_df=pd.read_csv('hw2_data/D_ICD_PROCEDURES.csv')
d_icd_procedures_df.to_sql("D_ICD_PROCEDURES", conn, index=False)

# Add drgcodes csv
drgcodes_df=pd.read_csv('hw2_data/DRGCODES.csv')
drgcodes_df.to_sql("DRGCODES", conn, index=False)

# Add icustays csv
icustays_df=pd.read_csv('hw2_data/ICUSTAYS.csv')
icustays_df.to_sql("ICUSTAYS", conn, index=False)

# Add patients csv
patients_df=pd.read_csv('hw2_data/PATIENTS.csv')
patients_df.to_sql("PATIENTS", conn, index=False)

# Add prescriptions csv
prescriptions_df=pd.read_csv('hw2_data/PRESCRIPTIONS.csv')
prescriptions_df.to_sql("PRESCRIPTIONS", conn, index=False)

# Add procedures_icd csv
procedures_icd=pd.read_csv('hw2_data/PROCEDURES_ICD.csv')
procedures_icd.to_sql("PROCEDURES_ICD", conn, index=False)
'''

'\n# Add admissions csv\nadmissions_df=pd.read_csv(\'hw2_data/ADMISSIONS.csv\')\nadmissions_df.to_sql("ADMISSIONS", conn, index=False)\n\n# Add d_icd_procedures csv\nd_icd_procedures_df=pd.read_csv(\'hw2_data/D_ICD_PROCEDURES.csv\')\nd_icd_procedures_df.to_sql("D_ICD_PROCEDURES", conn, index=False)\n\n# Add drgcodes csv\ndrgcodes_df=pd.read_csv(\'hw2_data/DRGCODES.csv\')\ndrgcodes_df.to_sql("DRGCODES", conn, index=False)\n\n# Add icustays csv\nicustays_df=pd.read_csv(\'hw2_data/ICUSTAYS.csv\')\nicustays_df.to_sql("ICUSTAYS", conn, index=False)\n\n# Add patients csv\npatients_df=pd.read_csv(\'hw2_data/PATIENTS.csv\')\npatients_df.to_sql("PATIENTS", conn, index=False)\n\n# Add prescriptions csv\nprescriptions_df=pd.read_csv(\'hw2_data/PRESCRIPTIONS.csv\')\nprescriptions_df.to_sql("PRESCRIPTIONS", conn, index=False)\n\n# Add procedures_icd csv\nprocedures_icd=pd.read_csv(\'hw2_data/PROCEDURES_ICD.csv\')\nprocedures_icd.to_sql("PROCEDURES_ICD", conn, index=False)\n'

In [4]:
# Close connection and reopen to get rid of .db.wal file
conn.close()
conn=duckdb.connect('hw2.db', read_only=False)

In [5]:
# All tables are loaded, we are now able to proceed with analysis
conn.sql('SHOW TABLES;')

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ ADMISSIONS       │
│ DRGCODES         │
│ D_ICD_PROCEDURES │
│ ICUSTAYS         │
│ PATIENTS         │
│ PRESCRIPTIONS    │
│ PROCEDURES_ICD   │
└──────────────────┘

Analysis Question 1: Create a summary of type of drugs and their total amount used by ethnicity. Report the top usage in each ethnicity group. You may have to make certain assumptions in calculating their total amount.

In [6]:
# First, create summary of type of drugs and total amount used by ethnicity
conn.sql(
    """
    SELECT ethnicity as ETHNICITY,drug as DRUG, count(*) as NUM_ORDERS
    FROM PRESCRIPTIONS 
    JOIN ADMISSIONS ON ADMISSIONS.hadm_id=prescriptions.hadm_id
    GROUP BY ethnicity, drug
    ORDER BY ethnicity;
    """
)

┌──────────────────────────────────────────────────────────┬──────────────────────────────────────────┬────────────┐
│                        ETHNICITY                         │                   DRUG                   │ NUM_ORDERS │
│                         varchar                          │                 varchar                  │   int64    │
├──────────────────────────────────────────────────────────┼──────────────────────────────────────────┼────────────┤
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ D5W                                      │          2 │
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ LeVETiracetam                            │          5 │
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ Phenylephrine                            │          1 │
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ Pantoprazole Sodium                      │          2 │
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ Bac

In [7]:
# Find top usage in each ethnicity
conn.sql(
    """
    WITH totals AS (
    SELECT ethnicity,drug, count(*) as NUM_ORDERS
        FROM PRESCRIPTIONS 
        JOIN ADMISSIONS ON ADMISSIONS.hadm_id=prescriptions.hadm_id
        GROUP BY ethnicity, drug),
    max_per_ethnicity AS (
        SELECT ethnicity, max(NUM_ORDERS) as MAX_AMOUNT
        From totals GROUP BY ethnicity)
    SELECT totals.ethnicity as ETHNICITY, drug as DRUG, MAX_AMOUNT as MAX_AMOUNT
    From totals
    JOIN max_per_ethnicity 
    ON totals.ethnicity = max_per_ethnicity.ethnicity
    AND totals.NUM_ORDERS=max_per_ethnicity.MAX_AMOUNT
    ORDER BY ETHNICITY; 
    """
)

┌──────────────────────────────────────────────────────────┬──────────────────────┬────────────┐
│                        ETHNICITY                         │         DRUG         │ MAX_AMOUNT │
│                         varchar                          │       varchar        │   int64    │
├──────────────────────────────────────────────────────────┼──────────────────────┼────────────┤
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ 5% Dextrose          │         27 │
│ ASIAN                                                    │ D5W                  │         27 │
│ BLACK/AFRICAN AMERICAN                                   │ Insulin              │         38 │
│ HISPANIC OR LATINO                                       │ 5% Dextrose          │         28 │
│ HISPANIC/LATINO - PUERTO RICAN                           │ 0.9% Sodium Chloride │         86 │
│ OTHER                                                    │ NS                   │         11 │
│ UNABLE TO OBTAIN            

Analysis Question 2: Create a summary of procedures performed on patients by age groups (<=19, 20-49, 50-79, >80). Report the top three procedures, along with the name of the procedures, performed in each age group.

In [92]:
conn.sql(
    """
    WITH actual_age AS (
    SELECT 
        date_diff('year', 
            CAST(PATIENTS.DOB AS DATE),
            CAST(ADMISSIONS.admittime AS DATE)) as AGE,
        D_ICD_PROCEDURES.short_title as Procedure,
        count(*) as procedure_count
    FROM PROCEDURES_ICD
    JOIN PATIENTS ON PATIENTS.subject_id=PROCEDURES_ICD.subject_id
    Join ADMISSIONS ON PROCEDURES_ICD.hadm_id=ADMISSIONS.hadm_id
    JOIN D_ICD_PROCEDURES ON PROCEDURES_ICD.icd9_code = D_ICD_PROCEDURES.icd9_code
    GROUP BY AGE, D_ICD_PROCEDURES.short_title),
    counts AS (
        Select CASE
                WHEN AGE <=19 THEN '<=19'
                WHEN AGE BETWEEN 20 AND 49 THEN '20-49'
                WHEN AGE BETWEEN 50 AND 79 THEN '50-79'
                ELSE '>80' END AS Age_Group,
            Procedure,
            SUM(procedure_count) as Total_Procedures,
        FROM actual_age
        WHERE AGE BETWEEN 0 AND 120
        GROUP BY Age_Group, procedure)
    Select age_group, Procedure, total_procedures FROM counts
    QUALIFY
    ROW_NUMBER() OVER (
        PARTITION BY age_group
        ORDER BY Total_Procedures DESC
    ) <= 3
    Order by
        CASE
            WHEN Age_Group = '<=19' THEN 1
            WHEN Age_Group = '20-49' THEN 2
            WHEN Age_Group = '50-79' THEN 3
            WHEN Age_Group = '>80' THEN 4
        END, total_procedures DESC;
    """
)

┌───────────┬──────────────────────────┬──────────────────┐
│ Age_Group │        Procedure         │ Total_Procedures │
│  varchar  │         varchar          │      int128      │
├───────────┼──────────────────────────┼──────────────────┤
│ <=19      │ Venous cath NEC          │                2 │
│ <=19      │ Vertebral fx repair      │                1 │
│ <=19      │ Interruption vena cava   │                1 │
│ 20-49     │ Venous cath NEC          │                9 │
│ 20-49     │ Entral infus nutrit sub  │                7 │
│ 20-49     │ Insert endotracheal tube │                6 │
│ 50-79     │ Venous cath NEC          │               25 │
│ 50-79     │ Entral infus nutrit sub  │               22 │
│ 50-79     │ Packed cell transfusion  │               13 │
│ >80       │ Venous cath NEC          │               16 │
│ >80       │ Packed cell transfusion  │               13 │
│ >80       │ Insert endotracheal tube │                8 │
├───────────┴──────────────────────────┴

Analysis Question 3: How long do patients stay in the ICU? Is there a difference in the ICU length of stay among gender or ethnicity?

In [9]:
# To start, analyze quantiles of how long patients stay in the ICU.
conn.sql(
    '''
    WITH STAY_LENGTHS AS (
    SELECT subject_id, date_diff('hour',
                                CAST(intime AS TIMESTAMP),
                                CAST(outtime AS TIMESTAMP)) as Hours FROM ICUSTAYS)
    SELECT 
        percentile_cont(0) WITHIN GROUP (ORDER BY Hours) as Min_Hours,
        percentile_cont(.25) WITHIN GROUP (ORDER BY Hours) as Q1_Hours,
        percentile_cont(.5) WITHIN GROUP (ORDER BY Hours) as Median_Hours,
        percentile_cont(.75) WITHIN GROUP (ORDER BY Hours) as Q3_Hours,
        percentile_cont(1) WITHIN GROUP (ORDER BY Hours) as Max_Hours,
    From STAY_LENGTHS
    '''
)

┌───────────┬──────────┬──────────────┬──────────┬───────────┐
│ Min_Hours │ Q1_Hours │ Median_Hours │ Q3_Hours │ Max_Hours │
│  double   │  double  │    double    │  double  │  double   │
├───────────┼──────────┼──────────────┼──────────┼───────────┤
│       3.0 │     29.5 │         50.5 │    104.0 │     849.0 │
└───────────┴──────────┴──────────────┴──────────┴───────────┘

In [10]:
# Next find average ICU length for each pair of gender/ethnicity
conn.sql(
    '''
    SELECT ADMISSIONS.ethnicity, PATIENTS.gender,
                        ROUND(AVG(date_diff('hour',
                        CAST(intime AS TIMESTAMP),
                        CAST(outtime AS TIMESTAMP))),2) as Average_Stay,
                        count(*) as Num_Records
    FROM ICUSTAYS
    JOIN ADMISSIONS on ADMISSIONS.hadm_id = ICUSTAYS.hadm_id
    JOIN PATIENTS on ADMISSIONS.subject_id = PATIENTS.subject_id
    GROUP BY ADMISSIONS.ethnicity, PATIENTS.gender
    ORDER BY ADMISSIONS.ethnicity, PATIENTS.gender;
    '''
)

┌──────────────────────────────────────────────────────────┬─────────┬──────────────┬─────────────┐
│                        ethnicity                         │ gender  │ Average_Stay │ Num_Records │
│                         varchar                          │ varchar │    double    │    int64    │
├──────────────────────────────────────────────────────────┼─────────┼──────────────┼─────────────┤
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ M       │        272.0 │           2 │
│ ASIAN                                                    │ F       │         16.0 │           1 │
│ ASIAN                                                    │ M       │        170.0 │           1 │
│ BLACK/AFRICAN AMERICAN                                   │ F       │        269.0 │           4 │
│ BLACK/AFRICAN AMERICAN                                   │ M       │        71.33 │           3 │
│ HISPANIC OR LATINO                                       │ F       │       179.33 │           3 │


#### Part 2

Question 1 - Connect to Cassandra

In [11]:
!curl https://certs.secureserver.net/repository/sf-class2-root.crt -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1468  100  1468    0     0   5886      0 --:--:-- --:--:-- --:--:--  5872


In [12]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# ssl setup
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('sf-class2-root.crt')  # change your file path for locating the certificate
ssl_context.verify_mode = CERT_REQUIRED

# boto3 session setup
boto_session = boto3.Session(region_name="us-east-2")  # this AWS credentials is specific to `us-east-2` region

/tmp/ipykernel_4974/332746716.py:7: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2)


In [13]:
# authorization setup with SigV4
auth_provider = SigV4AuthProvider(boto_session)

In [14]:
#cluster setup 
cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], 
                  ssl_context=ssl_context, 
                  auth_provider=auth_provider, 
                  port=9142)  # TLS only communicates on port 9142


In [15]:
# establishing connection to Keyspace
session = cluster.connect()

In [16]:
# Create custom hw2 keyspace
r = session.execute('''
    CREATE KEYSPACE IF NOT EXISTS atb2199_ks 
    WITH replication = {'class': 'SingleRegionStrategy'};
    ''')
print(r.current_rows)

[]


In [36]:
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra import ConsistencyLevel


# Define execution profile with LOCAL_QUORUM
execution_profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)

# Cluster setup with correct profile
cluster = Cluster(
    ['cassandra.us-east-2.amazonaws.com'],
    ssl_context=ssl_context,
    auth_provider=auth_provider,
    port=9142,
    execution_profiles={EXEC_PROFILE_DEFAULT: execution_profile}
)

# establishing connection to Keyspace
session = cluster.connect()
session.set_keyspace('atb2199_ks')  # Replace with your keyspace

Question 2 - Statement signed in narrative

Question 3

Analysis Question 1: Create a summary of type of drugs and their total amount used by ethnicity. Report the top usage in each ethnicity group. You may have to make certain assumptions in calculating their total amount.

Part A

In [46]:
session.execute("DROP TABLE IF EXISTS question_1")

In [47]:
session.execute("""
CREATE TABLE IF NOT EXISTS question_1 (
    id UUID PRIMARY KEY,
    ethnicity text,
    drug text
)
""")

Part B

In [48]:
import uuid
import csv

# Create merge csv files together using pandas
drugs_df=pd.read_csv('hw2_data/PRESCRIPTIONS.csv', usecols=['hadm_id','drug'])
admissions_df=pd.read_csv('hw2_data/ADMISSIONS.csv', usecols=['hadm_id','ethnicity'])
df1=drugs_df.merge(admissions_df, on='hadm_id', how='inner')

# Upload information to Cassandra
for i in range(df1.shape[0]): 
    session.execute("""
        INSERT INTO question_1 (id, ethnicity, drug)
        VALUES (%s, %s, %s)
    """, (uuid.uuid4(), df1.loc[i,'ethnicity'], df1.loc[i, 'drug']))

Part C

In [49]:
# Extract data from cassandra, then perform post-extraction analysis using python to aggregate.
rows=session.execute("SELECT ethnicity, drug FROM question_1")
extracted_df=pd.DataFrame(rows, columns=['ethnicity', 'drug'])

# Create counts per prescription using pandas.groupby function
counts=extracted_df.groupby(['ethnicity', 'drug']).size().reset_index(name='total_orders')

# Pick drug with max total_orders in each ethnicity
# First, find id number for each maximum grouped over ethnicity using idxmax
max_ids=counts.groupby('ethnicity')['total_orders'].idxmax()
# Then, create a dataframe using these ids
q1_df=counts.loc[max_ids]

Part D

In [50]:
q1_df

,ethnicity,drug,total_orders
2,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,5% Dextrose,27
89,ASIAN,D5W,27
205,BLACK/AFRICAN AMERICAN,Insulin,38
294,HISPANIC OR LATINO,5% Dextrose,28
377,HISPANIC/LATINO - PUERTO RICAN,0.9% Sodium Chloride,86
555,OTHER,NS,11
572,UNABLE TO OBTAIN,0.9% Sodium Chloride,28
655,UNKNOWN/NOT SPECIFIED,D5W,37
1152,WHITE,Potassium Chloride,381


Analysis Question 2: Create a summary of procedures performed on patients by age groups (<=19, 20-49, 50-79, >80). Report the top three procedures, along with the name of the procedures, performed in each age group.

Part A

In [51]:
session.execute("DROP TABLE IF EXISTS question_2")

In [54]:
session.execute("""
CREATE TABLE IF NOT EXISTS question_2 (
    id UUID PRIMARY KEY,
    hadm_id int,
    subject_id int,
    short_title text,
    dob timestamp,
    admittime timestamp,
)
""")

Part B

In [62]:
# Load csvs
proc_icd=pd.read_csv('hw2_data/PROCEDURES_ICD.csv', usecols=['subject_id','hadm_id','icd9_code'])
d_icd=pd.read_csv('hw2_data/D_ICD_PROCEDURES.csv', usecols=['icd9_code','short_title'])
admissions=pd.read_csv('hw2_data/ADMISSIONS.csv', usecols=['subject_id','hadm_id','admittime'])
patients=pd.read_csv('hw2_data/PATIENTS.csv', usecols=['subject_id','dob'])

# Merge dfs together
df=proc_icd.merge(d_icd, on='icd9_code',how='left')
df=df.merge(admissions, on=['subject_id','hadm_id'], how='left')
df2=df.merge(patients, on='subject_id',how='left')

# Fill nan values with a string, was getting a type error
df2['short_title'] = df2['short_title'].fillna('UNKNOWN_PROCEDURE')

# Insert into Cassandra
for i in range(df2.shape[0]): 
    session.execute("""
        INSERT INTO question_2 (id, hadm_id, subject_id, short_title, dob, admittime)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (uuid.uuid4(), df2.loc[i,'hadm_id'], df2.loc[i, 'subject_id'], df2.loc[i, 'short_title'], df2.loc[i, 'dob'], df2.loc[i, 'admittime']))
    

Part C

In [93]:
# Extract data
rows=session.execute("""SELECT hadm_id, short_title AS procedure_name, admittime, dob FROM question_2""")
extracted_df2=pd.DataFrame(rows, columns=['hadm_id','procedure_name','admittime','dob'])

# Create age column
extracted_df2['admittime']=pd.to_datetime(extracted_df2['admittime'])
extracted_df2['dob']=pd.to_datetime(extracted_df2['dob'])
# For the age column, first set both admittime and dob as numpy arrays, then subtract, then turn into years. 
# Was getting int overflow errors before
extracted_df2['age']=((extracted_df2['admittime'].to_numpy(dtype='datetime64[D]')
                      -extracted_df2['dob'].to_numpy(dtype='datetime64[D]')).astype(int))//365

# Now, we can turn the age col into age buckets
# Define a helper function
def helper_bucket(age):
    if age <= 19:
        return '<=19'
    elif age <= 49:
        return '20-49'
    elif age <= 79:
        return '50-79'
    else:
        return '>80'
extracted_df2['age_group']=extracted_df2['age'].map(helper_bucket)

# Now, we can aggregate and pick the top three, similar to the aggregation done in analysis question 1
# First, get a column of counts grouped by age group and procedure name
counts=extracted_df2.groupby(['age_group','procedure_name']).size().reset_index(name='total_count')

# Now, select the top three from counts using .head(i), which displays the top i entries for each groupby. Sort_values makes this easy
top3=counts.sort_values(['age_group','total_count'], ascending=[True, False]).groupby('age_group').head(3)

Part D

In [94]:
top3

,age_group,procedure_name,total_count
47,20-49,Venous cath NEC,9
13,20-49,Entral infus nutrit sub,7
9,20-49,Cont inv mec ven 96+ hrs,6
148,50-79,Venous cath NEC,26
80,50-79,Entral infus nutrit sub,22
111,50-79,Packed cell transfusion,13
166,<=19,Venous cath NEC,2
149,<=19,Applic ext fix dev-femur,1
150,<=19,Atlas-axis fusion,1
238,>80,Venous cath NEC,19


Analysis Question 3: How long do patients stay in the ICU? Is there a difference in the ICU length of stay among gender or ethnicity?

Part A

In [95]:
session.execute("DROP TABLE IF EXISTS question_3")

In [97]:
session.execute("""
CREATE TABLE IF NOT EXISTS question_3 (
    id UUID PRIMARY KEY,
    hadm_id int,
    subject_id int,
    intime timestamp,
    outtime timestamp,
    ethnicity text,
    gender text
)
""")

Part B

In [100]:
# Upload csv files
icustays=pd.read_csv('hw2_data/ICUSTAYS.csv', usecols=['subject_id','hadm_id','intime','outtime'])
admissions=pd.read_csv('hw2_data/ADMISSIONS.csv', usecols=['hadm_id','ethnicity'])
patients=pd.read_csv('hw2_data/PATIENTS.csv', usecols=['subject_id','gender'])

# Merge dfs together
df3=icustays.merge(admissions, on='hadm_id',how='left').merge(patients, on='subject_id',how='left')

# Insert into Cassandra
for i in range(df3.shape[0]): 
    session.execute("""
        INSERT INTO question_3 (id, hadm_id, subject_id, intime, outtime, ethnicity, gender)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (uuid.uuid4(), df3.loc[i,'hadm_id'], df3.loc[i, 'subject_id'], df3.loc[i, 'intime'], 
          df3.loc[i, 'outtime'], df3.loc[i, 'ethnicity'], df3.loc[i, 'gender']))

Part C

In [109]:
# Extract data from Cassandra
rows=session.execute("""SELECT intime, outtime, ethnicity, gender FROM question_3""")
extracted_df3=pd.DataFrame(rows, columns=['intime','outtime','ethnicity','gender'])

# Compute hours column in python, subtracting intime from outtime
extracted_df3['hours']=(pd.to_datetime(extracted_df3['outtime'])-
                        pd.to_datetime(extracted_df3['intime'])).dt.total_seconds()/3600

# First, create percentile dataframe from Part 1 using np.percentile
import numpy as np
percentiles=np.percentile(extracted_df3['hours'], [0,25,50,75,100])
percentile_df=pd.DataFrame({
    'Metric': ['Min','Q1','Median','Q3','Max'],
    'Value': percentiles})

# Next, create dataframe that shows average icu stay length for each combination of gender and ethnicity. 
average_df3=extracted_df3[['hours','ethnicity','gender']].groupby(['ethnicity','gender']).mean()

Part D

In [107]:
percentile_df

,Metric,Value
0,Min,2.542222
1,Q1,29.604097
2,Median,50.674722
3,Q3,103.897500
4,Max,849.756389


In [110]:
average_df3

hours
ethnicity                                          gender            
AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNI... M       272.091806
ASIAN                                              F        15.907222
                                                   M       170.815278
BLACK/AFRICAN AMERICAN                             F       268.829792
                                                   M        71.453889
HISPANIC OR LATINO                                 F       179.031296
HISPANIC/LATINO - PUERTO RICAN                     M        77.833481
OTHER                                              F        32.067500
                                                   M         2.542222
UNABLE TO OBTAIN                                   M       320.568889
UNKNOWN/NOT SPECIFIED                              F       133.032407
                                                   M        51.491389
WHITE                                              F       124.701162
                                                   M        75.693299